In [9]:
!pip install transformers -q
!pip install langchain[docarray]  -q
!pip install docarray  -q
!pip install pypdf  -q
!pip install langchain_huggingface  -q
!pip install bitsandbytes  -q
!pip install langchain-community -q
!pip install sentence-transformers -q

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-2-9b-it", use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/gemma-2-9b-it",
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)

text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Device set to use cuda:0


In [26]:
from langchain_huggingface import HuggingFaceEmbeddings


embedding_name = 'sbunlp/fabert'
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_name,
    # model_kwargs={"trust_remote_code": True},
)

No sentence-transformers model found with name sbunlp/fabert. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at sbunlp/fabert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from langchain_huggingface.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=text_gen)

def apply_chat_template_and_response(prompt):
    messages = [
    {'role': 'user', 'content': prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    return llm.invoke(text).replace(text, '')

In [16]:
print(apply_chat_template_and_response("اسمت چیه؟"))

C:\Users\GitiAI\anaconda3\envs\py310\lib\site-packages\transformers\integrations\sdpa_attention.py:96: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


من Gemma هستم، یک مدل زبان بزرگ هسته باز.



In [17]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
response_from_model = apply_chat_template_and_response("Give me an inspirational quote in persian")
parsed_response = parser.parse(response_from_model)
print(parsed_response)

**"چشمتان بَرِ بادِ امید باشد تا دلتان بَرِ دریای محبت."**

**Pronunciation:**

cheshmatan bare badi omid bashad ta deltan barey daryaye mohabat

**Translation:**

May your eyes be filled with the


In [20]:
from langchain.prompts import PromptTemplate

template = """
You are a helpful and knowledgeable AI assistant. Use only the information retrieved from the documents to answer the user's question in Persian (Farsi). If the answer is not found in the retrieved context, respond with: "متاسفانه اطلاعاتی در این مورد ندارم." Do not use your own knowledge beyond the provided context. Be accurate, clear, and polite. Never mention the documents or the retrieval process in your response. Just respond naturally in Persian.
Context: {context}

Question: {question}

Answer:

"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nYou are a helpful and knowledgeable AI assistant. Use only the information retrieved from the documents to answer the user\'s question in Persian (Farsi). If the answer is not found in the retrieved context, respond with: "متاسفانه اطلاعاتی در این مورد ندارم." Do not use your own knowledge beyond the provided context. Be accurate, clear, and polite. Never mention the documents or the retrieval process in your response. Just respond naturally in Persian.\nContext: Here is some context\n\nQuestion: Here is a question\n\nAnswer:\n\n'

In [21]:
context = "من زهرا هستم و تهران زندگی میکنم."
formatted_prompt = prompt.format(context = context, question="اسم من چیه؟")
response_from_model = apply_chat_template_and_response(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response.replace(formatted_prompt, ""))

زهرا 



In [22]:
formatted_prompt = prompt.format(context=context, question="من چند سالمه؟")
response_from_model = apply_chat_template_and_response(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response.replace(formatted_prompt, ""))

متاسفانه اطلاعاتی در این مورد ندارم. 



In [23]:
!curl -L -o  sample_doc.pdf "https://github.com/zdashti/testdoc/blob/main/CreativeThinking.pdf"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 30134    0 30134    0     0  28057      0 --:--:--  0:00:01 --:--:-- 28057
100  179k    0  179k    0     0   131k      0 --:--:--  0:00:01 --:--:--  131k


In [24]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader("CreativeThinking.pdf")
pages = loader.load_and_split()
#pages = loader.load()
pages

[Document(metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-10-11T09:27:35+03:30', 'author': 'GitiAI', 'moddate': '2025-10-11T09:27:35+03:30', 'source': 'CreativeThinking.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='تفکر خلاق و نقش آن در پیشرفت انسان \nتفکر خلاق، یکی از ارزشمندترین تواناییهای ذهن انسان است؛ قابلیتی که باعث می شود ما بتوانیم از مرزهای عادی فکر کردن عبور کنیم و \nراهحلهایی تازه برای مسائل پیدا کنیم. انسان از ابتدای تاریخ، به واسطه ی همین قدرت تفکر خلاق توانسته است ابزار بسازد، فناوری خلق کند، \nو جهان پیرامون خود را دگرگون سازد . \nاما خلاقیت تنها به هنرمندان یا مخترعان محدود نیست. در واقع، هر فردی در زندگی روزمره خود نیازمند تفکر خلاق است؛ از حل مشکلات  \nشخصی گرفته تا تصمیمگیریهای کاری و اجتماعی. خلاقیت یعنی دیدن چیزها از زاویه ای متفاوت، شکستن عادتهای فکری و تصور چیزی \nکه هنوز وجود ندارد. \n \nخلاقیت ذاتی است یا اکتسابی؟ \nبسیاری از مردم تصور میکنند که خلاقیت فقط در وجود عدهای خاص نهفته است،

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
text_documents = text_splitter.split_documents(pages)[:5]

pages

[Document(metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2025-10-11T09:27:35+03:30', 'author': 'GitiAI', 'moddate': '2025-10-11T09:27:35+03:30', 'source': 'CreativeThinking.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='تفکر خلاق و نقش آن در پیشرفت انسان \nتفکر خلاق، یکی از ارزشمندترین تواناییهای ذهن انسان است؛ قابلیتی که باعث می شود ما بتوانیم از مرزهای عادی فکر کردن عبور کنیم و \nراهحلهایی تازه برای مسائل پیدا کنیم. انسان از ابتدای تاریخ، به واسطه ی همین قدرت تفکر خلاق توانسته است ابزار بسازد، فناوری خلق کند، \nو جهان پیرامون خود را دگرگون سازد . \nاما خلاقیت تنها به هنرمندان یا مخترعان محدود نیست. در واقع، هر فردی در زندگی روزمره خود نیازمند تفکر خلاق است؛ از حل مشکلات  \nشخصی گرفته تا تصمیمگیریهای کاری و اجتماعی. خلاقیت یعنی دیدن چیزها از زاویه ای متفاوت، شکستن عادتهای فکری و تصور چیزی \nکه هنوز وجود ندارد. \n \nخلاقیت ذاتی است یا اکتسابی؟ \nبسیاری از مردم تصور میکنند که خلاقیت فقط در وجود عدهای خاص نهفته است،

In [27]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(text_documents, embedding=embeddings)

C:\Users\GitiAI\anaconda3\envs\py310\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [32]:
questions = [
    "تفکر خلاق به چه معناست؟",
    "آیا خلاقیت ذاتی است یا می‌توان آن را یاد گرفت؟",
    "یکی از موانع اصلی خلاقیت چیست؟",
    "چرا شکست در مسیر خلاقیت اهمیت دارد؟",
    "نقش محیط در شکوفایی خلاقیت چیست؟",
    "چند روش برای پرورش تفکر خلاق نام ببر.",
    "چرا خلاقیت در دنیای کار امروز اهمیت دارد؟",
    "نقش نظام آموزشی در پرورش خلاقیت چیست؟",
    "رابطه‌ی خلاقیت و فناوری چگونه است؟",
    "جامعه‌ی خلاق چه ویژگی‌هایی دارد؟"
]

retriever = vectorstore.as_retriever()

for question in questions:
    retrieved_context = retriever.invoke(question)
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = apply_chat_template_and_response(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Question: تفکر خلاق به چه معناست؟
Answer: تفکر خلاق یعنی ایمان به اینکه همیشه راهی هست، حتی اگر هنوز پیدایش نکرده باشیم. 


Question: آیا خلاقیت ذاتی است یا می‌توان آن را یاد گرفت؟
Answer: بسیاری از مردم تصور میکنند که خلاقیت فقط در وجود عده ای خاص نهفته است، اما پژوهش های روانشناسی نشان داده اند که همه ی انسانها از نوعی ظرفیت خلاق برخوردارند. تفاوت در این است که برخی افراد این توانایی را پرورش می دهند، در

Question: یکی از موانع اصلی خلاقیت چیست؟
Answer: یکی از بزرگترین دشمنان خلاقیت، ترس از اشتباه است. 


Question: چرا شکست در مسیر خلاقیت اهمیت دارد؟
Answer: ایده‌های بزرگ معمولاً از دل شکست‌های متعدد بیرون می‌آیند. 


Question: نقش محیط در شکوفایی خلاقیت چیست؟
Answer: یک محیط بسته و غیر انعطاف پذیر مانع از شکوفایی خلاقیت می‌شود. وقتی در فضایی کار یا زندگی می‌کنیم که در آن تفاوت نظرها پذیرفته نمی‌شود، ذهن به مرور حالت تدافعی می‌گیرد و از نوآوری دور می‌شود. برای شکوفایی خلاقیت،

Question: چند روش برای پرورش تفکر خلاق نام ببر.
Answer: در اینجا چند روش برای پرورش تفکر خلاق آورده شده است:

In [33]:
while True:
    print("Say 'exit' or 'quit' to exit the loop")
    question = input('User question: ')
    print(f"Question: {question}")
    if question.lower() in ["exit", "quit"]:
        print("Exiting the conversation. Goodbye!")
        break
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = apply_chat_template_and_response(formatted_prompt)
    parsed_response = parser.parse(response_from_model)
    print(f"Answer: {parsed_response}")
    print()

Say 'exit' or 'quit' to exit the loop


User question:  سلام


Question: سلام
Answer: سلام 


Say 'exit' or 'quit' to exit the loop


User question:  بر اساس متن، تفکر خلاق چه نقشی در زندگی فردی و شغلی انسان دارد؟


Question: بر اساس متن، تفکر خلاق چه نقشی در زندگی فردی و شغلی انسان دارد؟
Answer: خلاقیت در زندگی و کار، یکی از مهمترین مهارتهای قرن بیست و یکم محسوب می شود. شرکت ها و سازمانهای بزرگ به دنبال افرادی هستند که بتوانند فراتر از چارچوبها فکر کنند. در حوزهی کارآفرینی، تفکر خلاق پایهی

Say 'exit' or 'quit' to exit the loop


User question:  قرمه سبزی چطوری پخته می شه؟


Question: قرمه سبزی چطوری پخته می شه؟
Answer: متاسفانه اطلاعاتی در این مورد ندارم. 


Say 'exit' or 'quit' to exit the loop


User question:  exit


Question: exit
Exiting the conversation. Goodbye!


In [34]:
embeddings2 = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

vectorstore2 = DocArrayInMemorySearch.from_documents(text_documents, embedding=embeddings2)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

C:\Users\GitiAI\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\GitiAI\.cache\huggingface\hub\models--intfloat--multilingual-e5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [36]:
questions = [
    "تفکر خلاق به چه معناست؟",
    "آیا خلاقیت ذاتی است یا می‌توان آن را یاد گرفت؟",
    "یکی از موانع اصلی خلاقیت چیست؟",
    "چرا شکست در مسیر خلاقیت اهمیت دارد؟",
    "نقش محیط در شکوفایی خلاقیت چیست؟",
    "چند روش برای پرورش تفکر خلاق نام ببر.",
    "چرا خلاقیت در دنیای کار امروز اهمیت دارد؟",
    "نقش نظام آموزشی در پرورش خلاقیت چیست؟",
    "رابطه‌ی خلاقیت و فناوری چگونه است؟",
    "جامعه‌ی خلاق چه ویژگی‌هایی دارد؟"
]

retriever2 = vectorstore2.as_retriever()

for question in questions:
    retrieved_context = retriever2.invoke(question)
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = apply_chat_template_and_response(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()

Question: تفکر خلاق به چه معناست؟
Answer: تفکر خلاق یعنی ایمان به اینکه همیشه راهی است  —even اگر هنوز پیدایش نکرده باشیم. 


Question: آیا خلاقیت ذاتی است یا می‌توان آن را یاد گرفت؟
Answer: خلاقیت ذاتی است، اما می توان آن را یاد گرفت و پرورش داد. 


Question: یکی از موانع اصلی خلاقیت چیست؟
Answer: یکی از بزرگترین دشمنان خلاقیت، ترس از اشتباه است.  


Question: چرا شکست در مسیر خلاقیت اهمیت دارد؟
Answer: ایده های بزرگ معمولا از دل شکست های متعدد بیرون می آیند. 


Question: نقش محیط در شکوفایی خلاقیت چیست؟
Answer: برای شکوفایی خلاقیت، باید در فضایی بود که آزادی اندیشه و گفت وگو وجود داشته باشد. وقتی در محیطی کار یا زندگی میکنیم که در آن تفاوت نظرها پذیرفته نمیشود، ذهن به مرور حالت تدافعی میگیرد و از نوآوری دور میشود. 




Question: چند روش برای پرورش تفکر خلاق نام ببر.
Answer: در این متن چند روش برای پرورش تفکر خلاق پیشنهاد شده است:

* پرسیدن سئوال های جدید 
* تجربه کردن مسیرهای ناآشنا 
* نوشتن ایده ها
* الهام گرفتن از دیگران
* استراحت ذهنی 


Question: چرا خلاقیت در دنیای کار امروز اهم

In [37]:
while True:
    print("Say 'exit' or 'quit' to exit the loop")
    question = input('User question: ')
    print(f"Question: {question}")
    if question.lower() in ["exit", "quit"]:
        print("Exiting the conversation. Goodbye!")
        break
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = apply_chat_template_and_response(formatted_prompt)
    parsed_response = parser.parse(response_from_model)
    print(f"Answer: {parsed_response}")
    print()

Say 'exit' or 'quit' to exit the loop


User question:  سلام


Question: سلام
Answer: سلام 


Say 'exit' or 'quit' to exit the loop


User question:  بر اساس متن، تفکر خلاق چه نقشی در زندگی فردی و شغلی انسان دارد؟


Question: بر اساس متن، تفکر خلاق چه نقشی در زندگی فردی و شغلی انسان دارد؟
Answer: در دنیای امروز، خلاقیت یکی از مهمترین مهارتهای قرن بیست و یکم محسوب میشود. شرکت ها و سازمانهای بزرگ به دنبال افرادی هستند که بتوانند فراتر از چارچوبها فکر کنند. در حوزهی کارآفرینی، تفکر خلاق پایهی اصلی

Say 'exit' or 'quit' to exit the loop


User question:  قرمه سبزی چطوری پخته می شه؟


Question: قرمه سبزی چطوری پخته می شه؟
Answer: متاسفانه اطلاعاتی در این مورد ندارم. 


Say 'exit' or 'quit' to exit the loop


User question:  exit


Question: exit
Exiting the conversation. Goodbye!
